<span style="font-size: 2em;">Gerekli kütüphanelerin indirilmesi</span>


In [1]:
#!pip install opencv-python

In [2]:
#!pip install deepface

In [3]:
#!pip install pyttsx3

In [4]:
#!pip install SpeechRecognition

In [5]:
#!pip install pygame

In [6]:
#!pip install tensorflow

In [7]:
#!pip install opencv-python --upgrade

In [8]:
#!pip install deepface --upgrade

In [9]:
#!pip install PyAudio

<span style="font-size: 2em;">huggingface pipeline</span>

In [10]:
from transformers import pipeline

In [11]:
def gptbot(user_input):  
    generator = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B', device=0)
    
    print("Chatbot'a hoş geldiniz! Çıkmak için 'çık' yazabilirsiniz.")
    
    while True:
        # Kullanıcıdan soru alma
        if user_input.lower() == "çık":
            print("Chatbot: Görüşmek üzere!")
            break
    
        # Modelden cevap üretme
        response = generator(
            user_input,
            max_length=50,
            do_sample=True,
            temperature=0.7,
            top_k=50
        )
        
        # Üretilen cevabı yazdırma
        print("Chatbot:", response[0]['generated_text'])
        break

<span style="font-size: 2em;">Chat-Bot fonksiyonları</span>

In [12]:
#!pip install tf-keras

In [13]:
import speech_recognition as sr
import pyttsx3
import datetime
import pygame
import webbrowser
import random
import requests
isim = 'yok'


günler = {
    "Monday": "Pazartesi",
    "Tuesday": "Salı",
    "Wednesday": "Çarşamba",
    "Thursday": "Perşembe",
    "Friday": "Cuma",
    "Saturday": "Cumartesi",
    "Sunday": "Pazar"
}
stop_bot = False
api_key = "8f39651fb818f04d1b08ef733c5b0711"

def speak(text):
    engine = pyttsx3.init()
    engine.setProperty('rate', 150)
    engine.setProperty('voice', 'turkish')
    engine.say(text)
    engine.runAndWait()

def play_number_game():
    secret_number = random.randint(1, 100)
    speak("Bir ile yüz arasında bir sayı tuttum. Tahmin et")

    while True:
        query = listen()
        if "çıkış" in query:
            speak(f"Oyun bitti. Doğru sayı {secret_number} idi.")
            break

        if "tahmin" in query:
            guess = int([int(s) for s in query.split() if s.isdigit()][0])
            if guess == secret_number:
                speak("Tebrikler! Doğru tahmin ettiniz.")
                break
            elif guess < secret_number:
                speak("Daha yüksek bir sayı söyleyin.")
            else:
                speak("Daha düşük bir sayı söyleyin.")
        else:
            speak("Lütfen bir sayı söyleyin veya çıkmak için 'çıkış' deyin.")
    
def get_weather(api_key):
    city_name = "Kirsehir"
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city_name,
        "appid": api_key,
        "units": "metric",  # Metrik birimlerde hava durumu almak için
        "lang": "tr"        # Türkçe dilinde hava durumu almak için
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        weather_desc = data["weather"][0]["description"]
        temp = data["main"]["temp"]
        result = f"Kırşehir'de hava durumu: {weather_desc}. Sıcaklık: {temp}°C."
        print(result)
        return result
    else:
        return "Hava durumu bilgisi alınamadı."

def play_music():
    pygame.init()
    pygame.mixer.music.load('music1.mp3')
    pygame.mixer.music.play()


def listen():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        recognizer.adjust_for_ambient_noise(source)
        print("Sizi dinliyorum...")
        audio = recognizer.listen(source)
        
    try:
        print("Ses algılanıyor, lütfen bekleyin...")
        text = recognizer.recognize_google(audio, language='tr-TR')
        print("Ses algılandı:", text)
        return text.lower()
    except sr.UnknownValueError:
        return "Anlaşılmadı"
    except sr.RequestError:
        return "Bağlantı hatası"

def chatbot_response(query):
    global stop_bot
    if "gpt aktive et" in query: #gpt bot calismasi
        gptbot("query")
    elif "nasılsın" in query:
        return "İyiyim, teşekkür ederim. Siz nasılsınız?"
    elif "bugün günlerden ne" in query:
        today_english = datetime.datetime.now().strftime("%A")
        today_turkish = günler.get(today_english, "Bilinmiyor")
        print(today_turkish)
        return f"Bugün {today_turkish}."
    elif "yarın günlerden ne" in query:
        tomorrow = (datetime.datetime.now() + datetime.timedelta(days=1)).strftime("%A")
        tomorrow_turkish = günler.get(tomorrow, "Bilinmiyor")
        return f"Yarın {tomorrow_turkish}."
    elif "hava durumu" in query:
        return get_weather(api_key)
    elif "favori şarkımı çal" in query:
        play_music()
        return "Favori şarkınız çalınıyor."
    elif "benimle oyun oyna" in query:
        return play_number_game()
    elif "benim adım ne" in query:
        return f"Senin adın {isim}."
    elif "durdur" in query:
        pygame.mixer.music.stop()
        return "Şarkı durduruldu."
    elif "kapat" in query:
        stop_bot = True
        return "Chatbot kapatılıyor."
    else:
        return "Üzgünüm, bu konuda yardımcı olamıyorum."

pygame 2.6.1 (SDL 2.28.4, Python 3.9.20)
Hello from the pygame community. https://www.pygame.org/contribute.html


<span style="font-size: 2em;">Yüz tanıma işlemleri</span>

In [14]:
import threading
import cv2
from deepface import DeepFace
import time
import pyttsx3
import speech_recognition as sr

stop_threads = threading.Event()
engine = pyttsx3.init()
is_spoken = False
stop_bot = False
bot_responded = False
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

counter = 0

reference_img1 = cv2.imread("ufukcandemirsoy.jpeg")
reference_img2 = cv2.imread("second.jpg")

face_match1 = False
face_match2 = False


def check_face1(frame):
    global face_match1
    try:
        if DeepFace.verify(frame, reference_img1.copy())['verified']:
            face_match1 = True
        else:
            face_match1 = False
            
    except ValueError:
        face_match1 = False

def check_face2(frame):
    global face_match2
    try:
        if DeepFace.verify(frame, reference_img2.copy())['verified']:
            face_match2 = True
            
        else:
            face_match2 = False
            
    except ValueError:
        face_match2 = False

def chatbot_thread_func():
    global stop_bot
    while not stop_bot:
        query = listen()
        response = chatbot_response(query)
        speak(response)
        if stop_bot:
            break

chatbot_thread = threading.Thread(target=chatbot_thread_func)

while True:
    bot_responded = False
    if stop_bot:
        stop_bot = False
        chatbot_thread.join()
        
    ret, frame = cap.read()

    if ret:
        if counter % 30 == 0:
            if not stop_threads.is_set():
                try:
                    threading.Thread(target=check_face1, args=(frame.copy(),)).start()
                    threading.Thread(target=check_face2, args=(frame.copy(),)).start()
                except ValueError:
                    pass
        counter += 1

        if face_match1:
            cv2.putText(frame, "Ufuk!", (20, 450), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 1)
            isim = 'Ufuk'
            #print(isim)
            if not chatbot_thread.is_alive():
                chatbot_thread = threading.Thread(target=chatbot_thread_func)
                chatbot_thread.start()
            
        else:
            stop_threads.clear()
            cv2.putText(frame, "eslesme yok 1", (20, 450), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 1)
            
        if face_match2:
            cv2.putText(frame, "Second!", (20, 400), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 1)
            isim = 'Second'
            if not chatbot_thread.is_alive():
                chatbot_thread = threading.Thread(target=chatbot_thread_func)
                chatbot_thread.start()
            
        else:
            cv2.putText(frame, "eslesme yok 2", (20, 400), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 1)
        
        cv2.imshow('video', frame)

    key = cv2.waitKey(1)
    if key == ord('q') or stop_bot:
        break

cv2.destroyAllWindows()
cap.release()


Exception in thread Thread-4:
Traceback (most recent call last):
  File "C:\Users\Ufuk\anaconda3\envs\environment2\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\Ufuk\anaconda3\envs\environment2\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\Ufuk\anaconda3\envs\environment2\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Ufuk\AppData\Local\Temp\ipykernel_21180\4001011120.py", line 29, in check_face1
  File "C:\Users\Ufuk\anaconda3\envs\environment2\lib\site-packages\deepface\DeepFace.py", line 150, in verify
    return verification.verify(
  File "C:\Users\Ufuk\anaconda3\envs\environment2\lib\site-packages\deepface\modules\verification.py", line 103, in verify
    model: FacialRecognition = modeling.build_model(
  File "C:\Users\Ufuk\anaconda3\envs\environment2\lib\site-packages\deepface\modules\modeling.py", line 96, in build_model
 

In [15]:
#!pip install pyowm